<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
print(response)

<Response [200]>


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
parameters = {'lat':'-33.87', 'lon':'151.21'}
print(parameters)

{'lat': '-33.87', 'lon': '151.21'}


The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

Print the response header:

In [6]:
print(response)

<Response [200]>


Print the content of the response (the data that the server returned):

In [7]:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1587597661, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 421, \n      "risetime": 1587619973\n    }, \n    {\n      "duration": 659, \n      "risetime": 1587625628\n    }, \n    {\n      "duration": 564, \n      "risetime": 1587631505\n    }, \n    {\n      "duration": 369, \n      "risetime": 1587637494\n    }, \n    {\n      "duration": 413, \n      "risetime": 1587643385\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
print(response.text)

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1587597661, 
    "latitude": -33.87, 
    "longitude": 151.21, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 421, 
      "risetime": 1587619973
    }, 
    {
      "duration": 659, 
      "risetime": 1587625628
    }, 
    {
      "duration": 564, 
      "risetime": 1587631505
    }, 
    {
      "duration": 369, 
      "risetime": 1587637494
    }, 
    {
      "duration": 413, 
      "risetime": 1587643385
    }
  ]
}



Print just the "content-type" value from the header:

In [9]:
print(type(response.content))
print(type(response.text))

<class 'bytes'>
<class 'str'>


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
response.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1587597661,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 421, 'risetime': 1587619973},
  {'duration': 659, 'risetime': 1587625628},
  {'duration': 564, 'risetime': 1587631505},
  {'duration': 369, 'risetime': 1587637494},
  {'duration': 413, 'risetime': 1587643385}]}

What kind of object did this give us?

In [11]:
print(type(response.json()))
print(type(12))

<class 'dict'>
<class 'int'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
iss_location = response.json()['response']
iss_location

[{'duration': 421, 'risetime': 1587619973},
 {'duration': 659, 'risetime': 1587625628},
 {'duration': 564, 'risetime': 1587631505},
 {'duration': 369, 'risetime': 1587637494},
 {'duration': 413, 'risetime': 1587643385}]

Now extract the `risetime` strings into an array called `srisetimes`:

In [13]:
srisetimes = [r['risetime'] for r in iss_location]
srisetimes

[1587619973, 1587625628, 1587631505, 1587637494, 1587643385]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [14]:
risetimes = [datetime.fromtimestamp(t) for t in srisetimes]
risetimes

[datetime.datetime(2020, 4, 23, 15, 32, 53),
 datetime.datetime(2020, 4, 23, 17, 7, 8),
 datetime.datetime(2020, 4, 23, 18, 45, 5),
 datetime.datetime(2020, 4, 23, 20, 24, 54),
 datetime.datetime(2020, 4, 23, 22, 3, 5)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [15]:
for t in risetimes:
    print(t.strftime('%d/%m/%y %I:%M'))

23/04/20 03:32
23/04/20 05:07
23/04/20 06:45
23/04/20 08:24
23/04/20 10:03


Here is an endpoint that tells us who is onboard:

In [16]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [17]:
response.content

b'{"number": 3, "message": "success", "people": [{"craft": "ISS", "name": "Chris Cassidy"}, {"craft": "ISS", "name": "Anatoly Ivanishin"}, {"craft": "ISS", "name": "Ivan Vagner"}]}'

In [25]:
from datetime import date
response.json()

{'number': 3,
 'message': 'success',
 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'},
  {'craft': 'ISS', 'name': 'Anatoly Ivanishin'},
  {'craft': 'ISS', 'name': 'Ivan Vagner'}]}

In [30]:
internationalSpaceStation = "http://api.open-notify.org/astros.json"
inSpace = requests.get(internationalSpaceStation).json()['number']
print("Today's date:", date.today())
print("The Number of Humans in Space Right Now:", inSpace)

Today's date: 2020-04-23
The Number of Humans in Space Right Now: 3


In [ ]:
people = requests.get(internationalSpaceStation).json()['people']
names = [n['name'] for n in people]
print("Names of People in Space")
print(names)

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
    if response.status_code == 200:
        return False, response.status_code
    else:
        return True, response.status_code
    
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
else:
    print('API call successful')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

>

>

>



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



